In [69]:
pip install geopandas

In [70]:
import pandas as pd
import numpy as np
import math
import geopandas
import json

In [71]:
#Reading the geojson files containing the Geometry Data
geo = geopandas.read_file('https://raw.githubusercontent.com/ujwalgurung1/ujwalgurung1.github.io/main/project/data/UKBoundaries.geojson')
geo

,OBJECTID,LAD20CD,LAD20NM,LAD20NMW,BNG_E,BNG_N,LONG,LAT,SHAPE_Length,SHAPE_Area,geometry
0,1,E06000001,Hartlepool,,447160,531474,-1.27018,54.676140,0.662107,0.013379,"MULTIPOLYGON (((-1.24224 54.72297, -1.24194 54..."
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.544670,0.459499,0.007672,"MULTIPOLYGON (((-1.19860 54.58287, -1.16664 54..."
2,3,E06000003,Redcar and Cleveland,,464361,519597,-1.00608,54.567520,1.048933,0.034501,"MULTIPOLYGON (((-0.79189 54.55824, -0.80042 54..."
3,4,E06000004,Stockton-on-Tees,,444940,518183,-1.30664,54.556911,1.078582,0.028519,"MULTIPOLYGON (((-1.19319 54.62905, -1.20018 54..."
4,5,E06000005,Darlington,,428029,515648,-1.56835,54.535339,1.129501,0.027620,"MULTIPOLYGON (((-1.43836 54.59508, -1.42333 54..."
...,...,...,...,...,...,...,...,...,...,...,...
374,375,W06000020,Torfaen,Torfaen,327459,200480,-3.05101,51.698360,0.798982,0.016353,"MULTIPOLYGON (((-2.95891 51.62878, -2.98933 51..."
375,376,W06000021,Monmouthshire,Sir Fynwy,337812,209231,-2.90280,51.778271,2.159787,0.111320,"MULTIPOLYGON (((-2.65040 51.82612, -2.66087 51..."
376,377,W06000022,Newport,Casnewydd,337897,187432,-2.89769,51.582310,1.018605,0.024982,"MULTIPOLYGON (((-2.82220 51.55392, -2.86658 51..."
377,378,W06000023,Powys,Powys,302329,273255,-3.43531,52.348640,6.034297,0.685780,"MULTIPOLYGON (((-3.14750 52.89017, -3.13552 52..."


In [72]:
#Cleaning: Keeping only the relevant columns 
geo = geo.rename(columns={"LAD20NM": "Region Name", "LAD20CD": "Area Code" })
geo = geo.drop(['LAD20NMW', 'OBJECTID'], axis=1)
geo

,Area Code,Region Name,BNG_E,BNG_N,LONG,LAT,SHAPE_Length,SHAPE_Area,geometry
0,E06000001,Hartlepool,447160,531474,-1.27018,54.676140,0.662107,0.013379,"MULTIPOLYGON (((-1.24224 54.72297, -1.24194 54..."
1,E06000002,Middlesbrough,451141,516887,-1.21099,54.544670,0.459499,0.007672,"MULTIPOLYGON (((-1.19860 54.58287, -1.16664 54..."
2,E06000003,Redcar and Cleveland,464361,519597,-1.00608,54.567520,1.048933,0.034501,"MULTIPOLYGON (((-0.79189 54.55824, -0.80042 54..."
3,E06000004,Stockton-on-Tees,444940,518183,-1.30664,54.556911,1.078582,0.028519,"MULTIPOLYGON (((-1.19319 54.62905, -1.20018 54..."
4,E06000005,Darlington,428029,515648,-1.56835,54.535339,1.129501,0.027620,"MULTIPOLYGON (((-1.43836 54.59508, -1.42333 54..."
...,...,...,...,...,...,...,...,...,...
374,W06000020,Torfaen,327459,200480,-3.05101,51.698360,0.798982,0.016353,"MULTIPOLYGON (((-2.95891 51.62878, -2.98933 51..."
375,W06000021,Monmouthshire,337812,209231,-2.90280,51.778271,2.159787,0.111320,"MULTIPOLYGON (((-2.65040 51.82612, -2.66087 51..."
376,W06000022,Newport,337897,187432,-2.89769,51.582310,1.018605,0.024982,"MULTIPOLYGON (((-2.82220 51.55392, -2.86658 51..."
377,W06000023,Powys,302329,273255,-3.43531,52.348640,6.034297,0.685780,"MULTIPOLYGON (((-3.14750 52.89017, -3.13552 52..."


In [73]:
#Caling the UK Land registry "API" for the price history
prices_data = pd.read_csv('http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Average-prices-2021-10.csv?utm_medium=GOV.UK&utm_source=datadownload&utm_campaign=average_price&utm_term=9.30_15_12_21')

In [74]:
#Keeping only the relevant date
prices_data = prices_data[prices_data['Date'].str.contains("2021-10-01")]
#Cleaning data from "API"
prices_data = prices_data.rename(columns={"Region_Name": "Region Name", "Area_Code": "Area Code" })
prices_data = prices_data.drop(['Monthly_Change', 'Annual_Change', 'Average_Price_SA','Average_Price_SA', 'Date'], axis=1)
prices_data['Average_Price'] = prices_data['Average_Price'].astype(float)
prices_data['Average_Price'] = prices_data['Average_Price'].apply(np.ceil) 
prices_data

,Region Name,Area Code,Average_Price
133566,England,E92000001,285114.0
133567,Northern Ireland,N92000001,159109.0
133568,Wales,W92000004,203224.0
133569,Scotland,S92000003,181392.0
133570,Inner London,E13000001,602501.0
...,...,...,...
133982,South Gloucestershire,E06000025,326397.0
133983,City of Plymouth,E06000026,204631.0
133984,England and Wales,K04000001,280701.0
133985,Great Britain,K03000001,271512.0


In [75]:
#Merging Geojson and Price data based on "Area Code"
merged = pd.merge(geo, prices_data, on='Area Code', how='left')
#Cleaning the data - renaming columns and dropping irrelevant columns
merged = merged.drop(['Area Code', 'Region Name_y', 'BNG_E', 'BNG_N', 'LONG', 'LAT', 'SHAPE_Length', 'SHAPE_Area'], axis=1)
merged = merged.rename(columns={"Region Name_y": "Region Name", "Average_Price": "Average Price (£)", "Region Name_x": "Region Name",})
merged

,Region Name,geometry,Average Price (£)
0,Hartlepool,"MULTIPOLYGON (((-1.24224 54.72297, -1.24194 54...",134659.0
1,Middlesbrough,"MULTIPOLYGON (((-1.19860 54.58287, -1.16664 54...",132815.0
2,Redcar and Cleveland,"MULTIPOLYGON (((-0.79189 54.55824, -0.80042 54...",141096.0
3,Stockton-on-Tees,"MULTIPOLYGON (((-1.19319 54.62905, -1.20018 54...",148789.0
4,Darlington,"MULTIPOLYGON (((-1.43836 54.59508, -1.42333 54...",146919.0
...,...,...,...
374,Torfaen,"MULTIPOLYGON (((-2.95891 51.62878, -2.98933 51...",182971.0
375,Monmouthshire,"MULTIPOLYGON (((-2.65040 51.82612, -2.66087 51...",324841.0
376,Newport,"MULTIPOLYGON (((-2.82220 51.55392, -2.86658 51...",212366.0
377,Powys,"MULTIPOLYGON (((-3.14750 52.89017, -3.13552 52...",215528.0


In [77]:
#Exporting the final file ready for Keplar
merged.to_file("UKHousePrices.geojson")  